In [3]:
from PIL import Image, ImageDraw
import numpy as np

# Load the image
image_path = '/workspaces/CSDNA/Images/csDNA_v7_100x_robot_v4_rayhan_3424_T1_cropped_locs_complete.png'
image = Image.open(image_path)

# Convert image to numpy array
image_array = np.array(image)

# Find all dots: assuming that the dots are the only non-black pixels
# We'll use a threshold to define "non-black", e.g., any pixel that is not (0, 0, 0)
threshold = 60
dots = np.where(np.all(image_array > threshold, axis=-1))

# Create an array of coordinates (x, y) for the dots
dot_coordinates = list(zip(dots[1], dots[0]))

# Create a drawing context
draw = ImageDraw.Draw(image)

# Define circle radius
radius = 1

# Draw circles around each detected dot
for x, y in dot_coordinates:
    # Define the bounding box for the circle
    left_up_point = (x - radius, y - radius)
    right_down_point = (x + radius, y + radius)
    two_point_box = [left_up_point, right_down_point]
    
    # Draw the circle in red
    draw.ellipse(two_point_box, outline="red", width=1)

# Save the edited image
edited_image_path = 'image_with_dots_circled.png'
image.save(edited_image_path)


Using the picked dots to generate rectangles around dots that create a rectangle of a minimum and maximum size

In [36]:
from PIL import Image, ImageDraw
import numpy as np

# Load the image
image_path = '/workspaces/CSDNA/Images/image1x3.png'
image = Image.open(image_path)

# Convert image to numpy array
image_array = np.array(image)

# Find all dots: assuming that the dots are the only non-black pixels
threshold = 40
dots = np.where(np.all(image_array > threshold, axis=-1))

# Create an array of coordinates (x, y) for the dots
dot_coordinates = list(zip(dots[1], dots[0]))

# Function to check if two dots are within a certain radius
def are_dots_close(dot1, dot2, radius=20):
    return (dot1[0] - dot2[0])**2 + (dot1[1] - dot2[1])**2 <= radius**2

# Create a drawing context
draw = ImageDraw.Draw(image)

# Visited list and groups list
visited = []
groups = []

# Define circle radius for individual dots
single_dot_radius = 1

# Define circle radius for a group of dots
group_radius = 50

# Find groups of close dots
for dot in dot_coordinates:
    # Skip if dot has been visited
    if dot in visited:
        continue
    
    # Find all dots within the radius of the current dot
    close_dots = [d for d in dot_coordinates if are_dots_close(dot, d)]
    
    # If there are at least 4 dots (including the current one), we consider it a group
    if len(close_dots) >= 4:
        groups.append(close_dots)
        for d in close_dots:
            if d not in visited:
                visited.append(d)

# Draw circles around groups
for group in groups:
    # Calculate the centroid of the group
    centroid = tuple(map(lambda y: sum(y) / float(len(y)), zip(*group)))
    left_up_point = (centroid[0] - group_radius, centroid[1] - group_radius)
    right_down_point = (centroid[0] + group_radius, centroid[1] + group_radius)
    two_point_box = [left_up_point, right_down_point]
    
    # Draw the circle in blue
    draw.ellipse(two_point_box, outline="blue", width=2)

# Save the edited image
edited_image_path = 'image_with_groups_circled.png'
image.save(edited_image_path)



In [6]:
from PIL import Image, ImageDraw
import numpy as np

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, i):
        if i != self.parent[i]:
            self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, i, j):
        root_i = self.find(i)
        root_j = self.find(j)
        if root_i == root_j:
            return
        if self.rank[root_i] < self.rank[root_j]:
            self.parent[root_i] = root_j
        elif self.rank[root_i] > self.rank[root_j]:
            self.parent[root_j] = root_i
        else:
            self.parent[root_j] = root_i
            self.rank[root_i] += 1

# Load the image
image_path = '/workspaces/CSDNA/Images/image1x5.png'


# Convert image to numpy array
image = Image.open(image_path)

# Convert image to numpy array
image_array = np.array(image)

# Find all dots: assuming that the dots are the only non-black pixels
threshold = 60
dots = np.where(np.all(image_array > threshold, axis=-1))

# Create an array of coordinates (x, y) for the dots
dot_coordinates = list(zip(dots[1], dots[0]))

# Function to check if two dots are within a certain radius
def are_dots_close(dot1, dot2, radius=20):
    return (dot1[0] - dot2[0])**2 + (dot1[1] - dot2[1])**2 <= radius**2

# Create a drawing context
draw = ImageDraw.Draw(image)

# Initialize UnionFind
uf = UnionFind(len(dot_coordinates))

# Merge close dots into the same group
for i, dot1 in enumerate(dot_coordinates):
    for j, dot2 in enumerate(dot_coordinates):
        if i != j and are_dots_close(dot1, dot2):
            uf.union(i, j)

# Find groups of close dots and draw circles around them
grouped_dots = {}
for i, dot in enumerate(dot_coordinates):
    root = uf.find(i)
    if root not in grouped_dots:
        grouped_dots[root] = []
    grouped_dots[root].append(dot)

# Define circle radius for a group of dots
group_radius = 50

print("dots found", len(dot_coordinates))
print("groups found", len(grouped_dots))
# Draw circles around groups
for group in grouped_dots.values():
    if len(group) >= 100:  # If there are at least 4 dots in a group
        # Calculate the centroid of the group
        centroid = tuple(map(lambda y: sum(y) / float(len(y)), zip(*group)))
        left_up_point = (centroid[0] - group_radius, centroid[1] - group_radius)
        right_down_point = (centroid[0] + group_radius, centroid[1] + group_radius)
        two_point_box = [left_up_point, right_down_point]
        
        # Draw the circle in blue
        draw.ellipse(two_point_box, outline="green", width=8)

print("groups found", len(grouped_dots))
# Save the edited image
edited_image_path = 'image_with_groups_corrected.png'
image.save(edited_image_path)


dots found 5810
groups found 30
groups found 30


In [20]:
import numpy as np
from PIL import Image, ImageDraw

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, i):
        if i != self.parent[i]:
            self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, i, j):
        root_i = self.find(i)
        root_j = self.find(j)
        if root_i == root_j:
            return
        if self.rank[root_i] < self.rank[root_j]:
            self.parent[root_i] = root_j
        elif self.rank[root_i] > self.rank[root_j]:
            self.parent[root_j] = root_i
        else:
            self.parent[root_j] = root_i
            self.rank[root_i] += 1

def are_dots_close(dot1, dot2, squared_radius=400):
    return (dot1[0] - dot2[0])**2 + (dot1[1] - dot2[1])**2 <= squared_radius

# Load the image
image_path = '/workspaces/CSDNA/Images/image1x5.png'
image = Image.open(image_path)

# Convert image to numpy array
image_array = np.array(image)

# Find all dots: assuming that the dots are the only non-black pixels
threshold = 60
dots = np.where(np.all(image_array > threshold, axis=-1))
dot_coordinates = list(zip(dots[1], dots[0]))

# Initialize UnionFind
uf = UnionFind(len(dot_coordinates))

# Merge close dots into the same group
for i, dot1 in enumerate(dot_coordinates):
    for j, dot2 in enumerate(dot_coordinates[i+1:], start=i+1):
        if are_dots_close(dot1, dot2):
            uf.union(i, j)

# Find groups of close dots and draw circles around them
grouped_dots = {}
for i, dot in enumerate(dot_coordinates):
    root = uf.find(i)
    if root not in grouped_dots:
        grouped_dots[root] = []
    grouped_dots[root].append(dot)

# Define circle radius for a group of dots
group_radius = 50

# Create a drawing context
draw = ImageDraw.Draw(image)
i = 0
# Draw circles around groups
for group in grouped_dots.values():
    if len(group) >= 100:  # If there are at least 100 dots in a group
        # Calculate the centroid of the group
        centroid = np.mean(group, axis=0)
        left_up_point = (centroid[0] - group_radius, centroid[1] - group_radius)
        right_down_point = (centroid[0] + group_radius, centroid[1] + group_radius)
        two_point_box = [tuple(left_up_point), tuple(right_down_point)]
        
        # Draw the circle in blue
        draw.ellipse(two_point_box, outline="green", width=8)
        i+= 1
        print("Ellipse: ", i)

print("groups found", len(grouped_dots))

# Save the edited image
edited_image_path = 'image_with_groups_optimized.png'
image.save(edited_image_path)


Ellipse:  1
Ellipse:  2
Ellipse:  3
Ellipse:  4
Ellipse:  5
Ellipse:  6
Ellipse:  7
Ellipse:  8
Ellipse:  9
Ellipse:  10
Ellipse:  11
Ellipse:  12
Ellipse:  13
Ellipse:  14
Ellipse:  15
Ellipse:  16
Ellipse:  17
Ellipse:  18
Ellipse:  19
groups found 30


In [18]:
import numpy as np
from PIL import Image, ImageDraw
from sklearn.cluster import DBSCAN

# Load the image
image_path = '/workspaces/CSDNA/Images/image1x5.png'
image = Image.open(image_path)

# Convert image to numpy array
image_array = np.array(image)

# Find all dots: assuming that the dots are the only non-black pixels
threshold = 60
dots = np.where(np.all(image_array > threshold, axis=-1))
dot_coordinates = np.array(list(zip(dots[1], dots[0])))

# Perform DBSCAN clustering
db = DBSCAN(eps=20, min_samples=7).fit(dot_coordinates)

# Get labels and unique cluster identifiers
labels = db.labels_
unique_labels = set(labels)

# Initialize a dictionary to store grouped dots
grouped_dots = {label: [] for label in unique_labels if label != -1}

# Group dots based on cluster labels
for label, dot in zip(labels, dot_coordinates):
    if label != -1:
        grouped_dots[label].append(dot)

# Define circle radius for a group of dots
group_radius = 50

# Create a drawing context
draw = ImageDraw.Draw(image)

i = 0
# Draw circles around groups
for group in grouped_dots.values():
    if len(group) >= 100:  # If there are at least 6 dots in a group
        # Calculate the centroid of the group
        centroid = np.mean(group, axis=0)
        left_up_point = (centroid[0] - group_radius, centroid[1] - group_radius)
        right_down_point = (centroid[0] + group_radius, centroid[1] + group_radius)
        two_point_box = [tuple(left_up_point), tuple(right_down_point)]
        
        # Draw the circle in green
        i+=1
        draw.ellipse(two_point_box, outline="green", width=8)
        print("Ellipse: ", i)

print("Total number of groups : ", len(grouped_dots))
# Save the edited image
edited_image_path = 'image_with_groups_dbscan.png'
image.save(edited_image_path)


Total number of groups :  28
